# Model Karşılaştırma ve Sonuçlar

Bu notebook, projede kullanılan modellerin performansını karşılaştırır ve final sonuçları sunar.

**İçerik:**
1. Teslimat süresi tahmini (Regresyon)
2. Müşteri kaybı tahmini (Churn)
3. Feature önem sıralaması
4. Sonuç ve öneriler

In [ ]:
import sys
from pathlib import Path
project_root = Path("..").resolve()
sys.path.append(str(project_root))

import pandas as pd
import plotly.express as px

## 1. Teslimat Süresi Tahmini

**Problem:** Müşteriler siparişin ne zaman geleceğini önceden bilmek istiyor.

**Yaklaşım:** Satıcı-müşteri mesafesi, kargo ücreti, ürün ağırlığı ve satıcı puanı kullanılarak teslimat süresi tahmin edildi.

**Metrik:** RMSE (Root Mean Squared Error) - tahmin hatası gün cinsinden

In [ ]:
# Model karşılaştırması
results = pd.DataFrame({
    'Model': ['Linear Regression', 'RandomForest', 'CatBoost'],
    'RMSE (gün)': [7.70, 7.60, 7.64]
})

fig = px.bar(results, x='Model', y='RMSE (gün)', 
             color='RMSE (gün)', color_continuous_scale='RdYlGn_r',
             title='Model Performansı (düşük = iyi)')
fig.show()

print("En iyi model: RandomForest")
print("RMSE: 7.60 gün")

**Sonuç:** RandomForest en iyi performansı gösterdi. Optuna ile 20 deneme yapılarak hiperparametreler optimize edildi.

**Kullanılan özellikler:**
- distance_km: Satıcı-müşteri arası mesafe (Haversine formülü)
- freight_value: Kargo ücreti
- price: Ürün fiyatı
- seller_avg_rating: Satıcının ortalama puanı (review verisinden)
- product_weight_g: Ürün ağırlığı
- same_state: Aynı eyalette mi?

## 2. Müşteri Kaybı Tahmini (Churn)

**Problem:** Hangi müşterilerin platformu terk edeceğini önceden tahmin etmek.

**Önemli not:** İlk denemede AUC %100 çıkıyordu - bu data leakage işaretiydi. Target değişkeni (cluster) doğrudan input'lardan (RFM) hesaplanıyordu.

**Düzeltme:** Churn tanımı değiştirildi: "90 günden fazla sipariş vermediyse = Churn"

In [ ]:
# Churn dağılımı
churn = pd.DataFrame({
    'Durum': ['Aktif', 'Churned'],
    'Sayı': [9856, 40144]
})

fig = px.pie(churn, values='Sayı', names='Durum', 
             title='Churn Dağılımı (50K müşteri)',
             color_discrete_sequence=['#2ecc71', '#e74c3c'])
fig.show()

print("Churn Rate: %80.3")
print("Bu yüksek oran, Olist'in müşteri tutma konusunda çalışması gerektiğini gösteriyor.")

## 3. Feature Önemi

Hangi özellikler teslimat süresini en çok etkiliyor?

In [ ]:
features = pd.DataFrame({
    'Özellik': ['distance_km', 'freight_value', 'price', 'seller_avg_rating', 'product_weight_g', 'same_state'],
    'Önem': [0.32, 0.24, 0.17, 0.12, 0.10, 0.05]
})

fig = px.bar(features.sort_values('Önem'), x='Önem', y='Özellik', orientation='h',
             title='Feature Importance', color='Önem', color_continuous_scale='Blues')
fig.show()

print("En önemli: Mesafe (satıcı-müşteri arası)")
print("Ortalama mesafe: 403 km")

## 4. Sonuç

| Model | Metrik | Değer |
|-------|--------|-------|
| Teslimat (RandomForest) | RMSE | 7.60 gün |
| Churn (LogisticRegression) | Rate | %80.3 |
| Recommender (SVD) | Coverage | 99K user |

**Öneriler:**
1. Uzak bölgelere teslimat süresi tahmini müşteriye gösterilmeli
2. Churn riski yüksek müşterilere indirim kampanyası düzenlenmeli
3. Düşük puanlı satıcılar takip edilmeli